In [6]:
import torch.nn as nn
import torch
import torch.nn.functional as F

import torch
import torch.nn as nn
import torchmetrics

from tqdm import tqdm 
from transformers import AutoModel, AutoConfig

import hydra
# from omegaconf import DictConfig

from clearml import Task, Logger

from datamodule import data, utils
from models import LSTM, CNN, Bertweet, Roberta

from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE = device

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer, BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# roberta_tokens = roberta_tokenizer("in desperate need of and i can not stress this enough spring break", padding='max_length', max_length=20, truncation=True, return_tensors='pt')
# print(roberta_tokens)  

# bert_tokens = bert_tokenizer("in desperate need of and i can not stress this enough spring break", padding='max_length', max_length=20, truncation=True, return_tensors='pt')
# print(bert_tokens) 




In [11]:

class BertweetRCNN(torch.nn.Module):
    """
    Bertweet Recurrent CNN
    """
    def __init__(self, output_dim, dropout):
        super(BertweetRCNN, self).__init__()
        
        config = AutoConfig.from_pretrained("vinai/bertweet-base")
        config.output_hidden_states = True
        self.bertweet = AutoModel.from_pretrained("vinai/bertweet-base", config)
        self.hidden_size = self.bertweet.config.hidden_size
        
        self.lstm = torch.nn.LSTM(self.hidden_size, 384, batch_first=True, bidirectional=True, dropout=dropout)
        self.W = torch.nn.Linear(self.hidden_size + 2*384, 768) # basically the hidden state * 2
        self.tanh = torch.nn.Tanh()
        self.fc = torch.nn.Linear(768, output_dim)

    def forward(self, input_ids, attention_mask, token_type_ids):
        
        # output_bertweet = self.bertweet(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0]
        # output_embedded = batch size, seq_len, embedding_dim
        output_lstm, _ = self.lstm(output_bertweet)
        # output_lstm = batch size, seq_len, 2*lstm_hidden_size
        output = torch.cat([output_lstm, output_bertweet], 2)
        # output = batch size, seq_len, embedding_dim + 2*hidden_size
        output = self.tanh(self.W(output)).transpose(1, 2)
        # output = batch size, seq_len, hidden_size_linear -> batch size, hidden_size_linear, seq_len
        output = F.max_pool1d(output, output.size(2)).squeeze(2)
        # output = batch size, hidden_size_linear
        output = self.fc(output)
        # output = batch size, output_dim
        return output

from torchviz import make_dot, make_dot_from_trace

train_iterator, valid_iterator, test_iterator = data.roberta_data_loader(sarc_path, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, True, 0, MAX_LEN, roberta_tokenizer, 97)
sample = next(iter(train_iterator))

ids = sample['ids'].to(DEVICE, dtype = torch.long)
mask = sample['mask'].to(DEVICE, dtype = torch.long)
token_type_ids = sample['token_type_ids'].to(DEVICE, dtype = torch.long)
model = BertweetRCNN(2, 0.2)
model.to(DEVICE)
# make_dot(model(ids, mask, token_type_ids))



Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertweetRCNN(
  (bertweet): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [12]:
print(model)

BertweetRCNN(
  (bertweet): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)

text = "The capital of France, " + tokenizer.mask_token + ",contains the Eiffel Tower."

input = tokenizer.encode_plus(text, return_tensors = "pt")

mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)

logits = model(**input)
logits = logits.logits

softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_word = torch.argmax(mask_word, dim=1)

print(tokenizer.decode(top_word))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


paris


In [ ]:
print(bert_tokenizer.convert_ids_to_tokens(7867))
print(bert_tokenizer.convert_ids_to_tokens(6719))
print(bert_tokenizer.convert_ids_to_tokens(3092))
print(bert_tokenizer.convert_ids_to_tokens(2866))
print(bert_tokenizer.convert_ids_to_tokens(2097))
print(bert_tokenizer.convert_ids_to_tokens(2022))
print(bert_tokenizer.convert_ids_to_tokens(2583))
print(bert_tokenizer.convert_ids_to_tokens(2000))


In [ ]:
from transformers  import BertConfig, BertForSequenceClassification
import os
import random
import re
import torch
import numpy as np

def seed_everything(seed: int):
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    

seed = 47
seed_everything(seed)


In [ ]:
train_iterator, valid_iterator, test_iterator = data.roberta_data_loader(sarc_path, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, True, 0, MAX_LEN, roberta_tokenizer, SEED)


In [ ]:
batch = next(iter(train_iterator))

# print(batch)    
ids = batch['ids'].to(DEVICE, dtype = torch.long)
mask = batch['mask'].to(DEVICE, dtype = torch.long)
token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
targets = batch['targets'].to(DEVICE, dtype = torch.long)

print(targets)
outputs1 = model1(ids, mask, token_type_ids)
print(outputs1)

In [ ]:

loss = bce_loss(outputs1, targets.unsqueeze(1).float())
print(loss.item())

_, predictions = torch.max(outputs1.data, dim = 1)
print(predictions)
acc1 = calcuate_accuracy(predictions, targets)
print(acc1/BATCH_SIZE_TRAIN)

print("MODEL 2")
loss = cross_entropy_loss(outputs2, targets)
print(loss.item())

_, predictions2 = torch.max(outputs2.data, dim = 1)
print(predictions2)
acc2 = calcuate_accuracy(predictions2, targets)
print(acc2/BATCH_SIZE_TRAIN)


In [ ]:
probs = nn.Softmax(dim=1) # Or logsoftmax
criterion = nn.CrossEntropyLoss() 

softmax_output = probs(outputs)
loss = bce_loss(outputs, targets)

print(softmax_output)
print(loss)

In [ ]:
bert_modell = BertClass(0.25)
bert_modell.to(DEVICE)
ids = bert_tokens['input_ids'].to(DEVICE, dtype = torch.long)
mask = bert_tokens['attention_mask'].to(DEVICE, dtype = torch.long)
token_type_ids = bert_tokens['token_type_ids'].to(DEVICE, dtype = torch.long)


In [ ]:
outputs = bert_modell(ids, mask, token_type_ids)
outputs

In [ ]:
temp = torch.rand(2, 3)
print(temp)
temp[:, 0]

In [ ]:
config = RobertaConfig.from_pretrained('roberta-base')
config.output_hidden_states = True
roberta_model = RobertaModel.from_pretrained("roberta-base", config)
# print(model)

config = BertConfig.from_pretrained('bert-base-cased')
config.output_hidden_states = True
bert_model = BertModel.from_pretrained('bert-base-cased', config)

In [ ]:
bert_output_0 = bert_model(bert_tokens['input_ids'], bert_tokens['attention_mask'])[0]
bert_output_1 = bert_model(bert_tokens['input_ids'], bert_tokens['attention_mask'])[1]
pooler = bert_output_0[:, 0]

# print(roberta_output_0)
print(bert_output_0.shape)
# print(roberta_output_1)
print(bert_output_1.shape)
# print(pooler)
print(pooler.shape)

In [ ]:
class RCNN(nn.Module):
    """
    Recurrent Convolutional Neural Networks for Text Classification (2015)
    """
    def __init__(self, vocab_size, embedding_dim, hidden_size, hidden_size_linear, class_num, dropout):
        super(RCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True, dropout=dropout)
        self.W = nn.Linear(embedding_dim + 2*hidden_size, hidden_size_linear)
        self.tanh = nn.Tanh()
        self.fc = nn.Linear(hidden_size_linear, class_num)

    def forward(self, x):
        # x = |bs, seq_len|
        x_emb = self.embedding(x)
        # x_emb = |bs, seq_len, embedding_dim|
        output, _ = self.lstm(x_emb)
        # output = |bs, seq_len, 2*hidden_size|
        output = torch.cat([output, x_emb], 2)
        # output = |bs, seq_len, embedding_dim + 2*hidden_size|
        output = self.tanh(self.W(output)).transpose(1, 2)
        # output = |bs, seq_len, hidden_size_linear| -> |bs, hidden_size_linear, seq_len|
        output = F.max_pool1d(output, output.size(2)).squeeze(2)
        # output = |bs, hidden_size_linear|
        output = self.fc(output)
        # output = |bs, class_num|
        return output

In [ ]:
model = RCNN(vocab_size=50265, embedding_dim=768, hidden_size=512, hidden_size_linear=512, class_num = 2, dropout = 0.1).to(device)

batch = next(iter(train_iterator))
        
ids = batch['ids'].to(DEVICE, dtype = torch.long)
mask = batch['mask'].to(DEVICE, dtype = torch.long)
token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
targets = batch['targets'].to(DEVICE, dtype = torch.long)

# print(ids)
# print(mask)
# print(token_type_ids)
print(targets)
print(ids)
# outputs = model(ids)
print(outputs)

In [4]:
SEED = 97
sarc_path = '/home/tegzes/Desktop/FL-Detection-Experiments/datamodule/isarcasm2022.csv'
BATCH_SIZE_TRAIN = 8
BATCH_SIZE_TEST = 4
MAX_LEN = 256
HIDDEN_DIM = 64
OUTPUT_DIM = 1
EMBEDDING_LENGTH = 300
N_LAYERS = 2
LEARNING_RATE = 1e-5
BIDIRECTIONAL = False
DROPOUT = 0.25
N_EPOCHS = 3
utils.seed_everything(SEED)


In [ ]:
train_iterator, valid_iterator, test_iterator = data.roberta_data_loader(sarc_path, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, True, 0, MAX_LEN, bert_tokenizer, SEED)


In [ ]:
batch = next(iter(train_iterator))

print(batch)    
ids = batch['ids'].to(DEVICE, dtype = torch.long)
mask = batch['mask'].to(DEVICE, dtype = torch.long)
token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
targets = batch['targets'].to(DEVICE, dtype = torch.long)

# print(ids)
# print(mask)
# print(token_type_ids)
print(targets)
outputs = model(ids, mask, token_type_ids)
print(outputs)
# _, predictions = torch.max(outputs.data, dim = 1)
# print(predictions)
# loss = cross_entropy_loss(outputs, targets)
# print(loss)

In [ ]:
cross_entropy_loss = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct



In [ ]:
# task = Task.init(project_name="FL Detection", task_name="Training")

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0
    
    # using torch metrics to calculate the metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    
    model.train()

    for batch_idx, batch in tqdm(enumerate(iterator, 0)):
        
        ids = batch['ids'].to(DEVICE, dtype = torch.long)
        mask = batch['mask'].to(DEVICE, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
        targets = batch['targets'].to(DEVICE, dtype = torch.long)
        # tweet_lens = batch['tweet_len']

        outputs = model(ids, mask, token_type_ids)#, tweet_lens)#.to('cpu'))
        # outputs = model(ids, mask, token_type_ids)

        # targets = targets.unsqueeze(1) # for BCEWithLogitsLoss criterion
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        _, predictions = torch.max(outputs.data, dim = 1)
        acc = calcuate_accuracy(predictions, targets)

        # loss = criterion(predictions, targets)
        # epoch_loss += loss.item()

        no_of_iterations += 1
        no_of_examples += targets.size(0)
                
        metric_acc.update(predictions, targets)
        metric_f1.update(outputs, targets)
        metric_f1_micro.update(outputs, targets)
        metric_f1_macro.update(outputs, targets)
        metric_precision.update(predictions, targets)
        metric_recall.update(predictions, targets)

        optimizer.zero_grad()
        loss.backward()
        # for GPU
        optimizer.step()

        # Logger.current_logger().report_scalar(
        #     "train", "loss", iteration = (epoch * len(iterator) + batch_idx), value = loss.item())
#----
        # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(batch['ids']), len(iterator),
        #         100. * batch_idx / len(iterator), loss.item()))
        

    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_examples
        
    acc_torch = metric_acc.compute()
    print(f"Training Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Training F1: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Training F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Training F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Training Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Training Recall: {recall}")
    
    print(f"Training Loss Epoch: {epoch_loss}")
  
    # Logger.current_logger().report_scalar(
    #     "train", "accuracy", iteration=epoch, value=acc_torch)

    
    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
    
    return epoch_loss, epoch_acc

# evaluation routine
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0    
    
   # torch metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    model.eval()
    
    with torch.no_grad():
    
        for _, batch in tqdm(enumerate(iterator, 0)):

            ids = batch['ids'].to(DEVICE, dtype = torch.long)
            mask = batch['mask'].to(DEVICE, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
            targets = batch['targets'].to(DEVICE, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids)
        
            _, predictions = torch.max(outputs.data, dim = 1)

            loss = criterion(outputs, targets)
            
            acc = calcuate_accuracy(predictions, targets)

            epoch_loss += loss.item()
    
            metric_acc.update(predictions, targets)
            metric_f1.update(outputs, targets)
            metric_f1_micro.update(outputs, targets)
            metric_f1_macro.update(outputs, targets)
            metric_precision.update(predictions, targets)
            metric_recall.update(predictions, targets)

            no_of_iterations += 1
            no_of_examples += targets.size(0)
    
    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_iterations #no_of_examples

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     epoch_loss, acc, len(iterator),
    #     100. * acc / len(iterator)))

    acc_torch = metric_acc.compute()
    print(f"Validation Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Validation F1 Validation: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Validation F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Validation F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Validation Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Validation Recall: {recall}")
    
    print(f"Validation Loss Epoch: {epoch_loss}")

    # clear ml
    # Logger.current_logger().report_scalar(
    #     "test", "loss", iteration=epoch, value=epoch_loss)
    # Logger.current_logger().report_scalar(
    #     "test", "accuracy", iteration=epoch, value=acc_torch)

    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
             
    return epoch_loss, epoch_acc


# experiment loop
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, cross_entropy_loss)
    valid_loss, valid_acc = evaluate(model, valid_iterator, cross_entropy_loss)
        
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


test_loss, test_acc = evaluate(model, test_iterator, cross_entropy_loss)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# task.close()

In [ ]:
bert = BertModel.from_pretrained('bert-base-uncased')

print(bert(**bert_tokens)[0].shape)
# pooler = hidden_state[:, 0]

In [ ]:

class BertClass(torch.nn.Module):
    def __init__(self):
        super(BertClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.25)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[1]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output


bert_model = BertClass()
bert_model.to(DEVICE)

cross_entropy_loss = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()
optimizer = torch.optim.Adam(bert_model.parameters(), lr = LEARNING_RATE)
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0
    
    # using torch metrics to calculate the metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    
    model.train()

    for batch_idx, batch in tqdm(enumerate(iterator, 0)):
        
        ids = batch['ids'].to(DEVICE, dtype = torch.long)
        mask = batch['mask'].to(DEVICE, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
        targets = batch['targets'].to(DEVICE, dtype = torch.long)
        # tweet_lens = batch['tweet_len']

#         outputs = model(ids)#, tweet_lens)#.to('cpu'))
        outputs = model(ids, mask, token_type_ids)

        # targets = targets.unsqueeze(1) # for BCEWithLogitsLoss criterion
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        _, predictions = torch.max(outputs.data, dim = 1)
        acc = calcuate_accuracy(predictions, targets)

        # loss = criterion(predictions, targets)
        # epoch_loss += loss.item()

        no_of_iterations += 1
        no_of_examples += targets.size(0)
                
        metric_acc.update(predictions, targets)
        metric_f1.update(outputs, targets)
        metric_f1_micro.update(outputs, targets)
        metric_f1_macro.update(outputs, targets)
        metric_precision.update(predictions, targets)
        metric_recall.update(predictions, targets)

        optimizer.zero_grad()
        loss.backward()
        # for GPU
        optimizer.step()

        # Logger.current_logger().report_scalar(
        #     "train", "loss", iteration = (epoch * len(iterator) + batch_idx), value = loss.item())
#----
        # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(batch['ids']), len(iterator),
        #         100. * batch_idx / len(iterator), loss.item()))
        

    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_examples
        
    acc_torch = metric_acc.compute()
    print(f"Training Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Training F1: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Training F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Training F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Training Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Training Recall: {recall}")
    
    print(f"Training Loss Epoch: {epoch_loss}")
  
    # Logger.current_logger().report_scalar(
    #     "train", "accuracy", iteration=epoch, value=acc_torch)

    
    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
    
    return epoch_loss, epoch_acc

# evaluation routine
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0    
    
   # torch metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    model.eval()
    
    with torch.no_grad():
    
        for _, batch in tqdm(enumerate(iterator, 0)):

            ids = batch['ids'].to(DEVICE, dtype = torch.long)
            mask = batch['mask'].to(DEVICE, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
            targets = batch['targets'].to(DEVICE, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids)
        
            _, predictions = torch.max(outputs.data, dim = 1)

            loss = criterion(outputs, targets)
            
            acc = calcuate_accuracy(predictions, targets)

            epoch_loss += loss.item()
    
            metric_acc.update(predictions, targets)
            metric_f1.update(outputs, targets)
            metric_f1_micro.update(outputs, targets)
            metric_f1_macro.update(outputs, targets)
            metric_precision.update(predictions, targets)
            metric_recall.update(predictions, targets)

            no_of_iterations += 1
            no_of_examples += targets.size(0)
    
    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_iterations #no_of_examples

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     epoch_loss, acc, len(iterator),
    #     100. * acc / len(iterator)))

    acc_torch = metric_acc.compute()
    print(f"Validation Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Validation F1 Validation: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Validation F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Validation F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Validation Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Validation Recall: {recall}")
    
    print(f"Validation Loss Epoch: {epoch_loss}")

    # clear ml
    # Logger.current_logger().report_scalar(
    #     "test", "loss", iteration=epoch, value=epoch_loss)
    # Logger.current_logger().report_scalar(
    #     "test", "accuracy", iteration=epoch, value=acc_torch)

    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
             
    return epoch_loss, epoch_acc


for epoch in range(3):

    train_loss, _ = train(bert_model, train_iterator, optimizer, cross_entropy_loss)
    valid_loss, _ = evaluate(bert_model, valid_iterator, cross_entropy_loss)
        
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} ')
    print(f'\t Val. Loss: {valid_loss:.3f} ')


test_loss, _ = evaluate(bert_model, test_iterator, cross_entropy_loss)
print(f'Test Loss: {test_loss:.3f} ')